In [1]:
import datasets

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
dataset = load_dataset("ai4privacy/pii-masking-300k")

Generating validation split: 100%|██████████| 47728/47728 [00:00<00:00, 126773.73 examples/s]


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 177677
    })
    validation: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 47728
    })
})

In [4]:
#document is rand id
#full_text is simply the text
#tokens is full text split by tokens and put into list
#traling white space is a boolean saying whether there is a space after that token
#labels is O if NA or label if it has a label

In [5]:
#up to 29908 are english

In [6]:
document_ids = dataset['train']['id'][:29908]

In [7]:
full_text = dataset['train']['source_text'][:29908]

In [8]:
span_labels = dataset['train']['span_labels'][:29908]

In [12]:
#let's use the spacy english tokenizer
# Construction 2
from spacy.lang.en import English
nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
tokenizer = nlp.tokenizer

ModuleNotFoundError: No module named 'spacy'

In [70]:
from ast import literal_eval

label_text = []
labels = set()
for i, spans in enumerate(span_labels):
    spans = literal_eval(spans)
    text = full_text[i]
    for span in spans:
        var_to_repl = text[span[0]:span[1]]
        tokens = [i.text for i in tokenizer(var_to_repl)]
        whitespaces = [i.whitespace_ for i in tokenizer(var_to_repl)]
        if len(tokens) > 1:
            new_tokens = []
            for ind, token in enumerate(tokens):
                if ind == 0:
                    new_tokens.append(f"{span[2]}_START")
                    new_tokens.append(whitespaces[ind])
                    labels.add(f"{span[2]}_START")
                elif ind == len(tokens)-1:
                    new_tokens.append(f"{span[2]}_END")
                    new_tokens.append(whitespaces[ind])
                    labels.add(f"{span[2]}_END")
                else:
                    new_tokens.append(f"{span[2]}_MIDDLE")
                    new_tokens.append(whitespaces[ind])
                    labels.add(f"{span[2]}_MIDDLE")
            repl = "".join(new_tokens)
            text = text[:span[0]] + repl + text[span[1]:]
        else:
            text = text[:span[0]] + span[2] + text[span[1]:]
            labels.add(span[2])
    label_text.append(text)

In [89]:
tokens = [[i.text for i in tokenizer(i)] for i in full_text]

In [72]:
trailing_whitespaces = [[bool(i.whitespace_) for i in tokenizer(i)] for i in full_text]

In [73]:
labels = list(labels)

In [75]:
len(labels)

111

In [81]:
label_tokens = [[str(i) for i in tokenizer(i)] for i in label_text]

In [77]:
for label in label_tokens:
    for i,j in enumerate(label):
        label[i] = 'O'
        for l in labels:
            if l in j:
                label[i] = l
                break


In [20]:
data = []

for i in range(len(document_ids)):
    data.append({
        'document': document_ids[i][:-1],
        'full_text': full_text[i],
        'tokens': tokens[i],
        'labels': label_tokens[i],
        'trailing_whitespaces': trailing_whitespaces[i]
    })


In [21]:
import json


with open("train.json", "w") as f:
    f.write(json.dumps(data))